# EXPLAIN & TEST FLOWCHART 

In [3]:
import pandas as pd

In [9]:
# Function to handle file upload and data reading
def upload_and_process_files(uploaded_files):
    """
    Function to upload and process the CSV files.

    Args:
    uploaded_files (list): List of file paths.

    Returns:
    pd.DataFrame: Combined DataFrame from all uploaded CSV files.
    """
    for file_path in uploaded_files:
        print(f"Processing file: {file_path}")
        if file_path.endswith(".csv"):
            try:
                # Try to read the CSV file
                df = pd.read_csv(file_path)
                all_data.append(df)
            except pd.errors.ParserError:
                # Handle CSV parsing error
                raise ValueError(f"Error: The file {file_path} is not in the correct format of a .csv file.")
    
    if all_data:
        # Combine all DataFrames into one
        df_combined = pd.concat(all_data, ignore_index=True)
        return df_combined
    else:
        return None

In [10]:
# Replace with actual file paths
uploaded_files = [r'C:\Users\ungdu\Downloads\Test Chatgpt\Test data.csv']  # Provide full file path(s)

df_combined = upload_and_process_files(uploaded_files)

if df_combined is not None:
    # Display the DataFrame in Jupyter notebook
    display(df_combined)
else:
    print("No data to display.")

Processing file: C:\Users\ungdu\Downloads\Test Chatgpt\Test data.csv


,Câu hỏi,Câu trả lời
0,Các quả có mùi vị như thế nào,Quả cam ngon. Quả táo dở. Quả chanh chua. Quả ...
1,Các quả có hình dáng như thế nào,"Quả cam có hình tròn. Quả táo có hình tròn, hơ..."


In [19]:
print(df_combined)

                             Câu hỏi  \
0      Các quả có mùi vị như thế nào   
1  Các quả có hình dáng như thế nào    

                                         Câu trả lời  \
0  Quả cam ngon. Quả táo dở. Quả chanh chua. Quả ...   
1  Quả cam có hình tròn. Quả táo có hình tròn, hơ...   

                                              Chunks  
0  [Quả cam ngon., Quả táo dở., Quả chanh chua., ...  
1  [Quả cam có hình tròn. Quả táo có hình tròn, h...  


In [16]:
# Display the column names to check for any discrepancies
print(df_combined.columns)

Index(['Câu hỏi ', 'Câu trả lời', 'Chunks'], dtype='object')


In [17]:
# Strip any extra spaces from the column names
df_combined.columns = df_combined.columns.str.strip()

# Print the columns again to check
print(df_combined.columns)

Index(['Câu hỏi', 'Câu trả lời', 'Chunks'], dtype='object')


In [18]:
# Apply the chunking process to the "Câu trả lời" column
df_combined["Chunks"] = df_combined["Câu trả lời"].apply(lambda x: chunker.split_text(x))

# Display the chunked results
for index, row in df_combined.iterrows():
    # Make sure that the column names match exactly
    print(f"\nCâu hỏi: {row['Câu hỏi']}")
    print("Chunks:")
    for i, chunk in enumerate(row["Chunks"]):
        print(f"  Chunk {i+1}: {chunk}")


Câu hỏi: Các quả có mùi vị như thế nào
Chunks:
  Chunk 1: Quả cam ngon.
  Chunk 2: Quả táo dở.
  Chunk 3: Quả chanh chua.
  Chunk 4: Quả mít to. Quả mít rất thơm nữa

Câu hỏi: Các quả có hình dáng như thế nào 
Chunks:
  Chunk 1: Quả cam có hình tròn. Quả táo có hình tròn, hơi nhỏ.
  Chunk 2: Quả chanh hình bầu dục.
  Chunk 3: Quả mít to dài có vỏ xù xì.
  Chunk 4: Quả mít có thể lấy gỗ


In [20]:
df = pd.read_csv(file_path, encoding='utf-8')

# CHUNKING

In [21]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Ensure that punkt is downloaded for sentence tokenization
nltk.download("punkt", quiet=True)

class SemanticChunker:
    def __init__(self, threshold=0.3, embedding_type="tfidf"):
        self.threshold = threshold
        self.embedding_type = embedding_type
    
    def embed_function(self, sentences):
        if self.embedding_type == "tfidf":
            vectorizer = TfidfVectorizer().fit_transform(sentences)
            return vectorizer.toarray()
        else:
            raise ValueError("Unsupported embedding type")
    
    def split_text(self, text):
        sentences = nltk.sent_tokenize(text)
        sentences = [item for item in sentences if item and item.strip()]
        if not len(sentences):
            return []
        vectors = self.embed_function(sentences)
        similarities = cosine_similarity(vectors)
        chunks = [[sentences[0]]]
        for i in range(1, len(sentences)):
            sim_score = similarities[i-1, i]
            if sim_score >= self.threshold:
                chunks[-1].append(sentences[i])
            else:
                chunks.append([sentences[i]])
        return [' '.join(chunk) for chunk in chunks]

# Function to handle file upload and data reading
def upload_and_process_files(uploaded_files):
    all_data = []
    for file_path in uploaded_files:
        print(f"Processing file: {file_path}")
        if file_path.endswith(".csv"):
            try:
                df = pd.read_csv(file_path, encoding='utf-8')  # Ensure proper encoding
                df.columns = df.columns.str.strip()  # Strip extra spaces from column names
                all_data.append(df)
            except pd.errors.ParserError:
                raise ValueError(f"Error: The file {file_path} is not in the correct format of a .csv file.")
    if all_data:
        df_combined = pd.concat(all_data, ignore_index=True)
        return df_combined
    else:
        return None

# Replace with actual file paths
uploaded_files = [r'C:\Users\ungdu\Downloads\Test Chatgpt\Test data.csv']  # Provide full file path(s)

# Upload and process the file
df_combined = upload_and_process_files(uploaded_files)

if df_combined is not None:
    # Display the DataFrame in Jupyter notebook
    print(df_combined.head())  # Check the first few rows

    # Initialize the SemanticChunker
    chunker = SemanticChunker(threshold=0.3)

    # Apply the chunking process to the "Câu trả lời" column
    df_combined["Chunks"] = df_combined["Câu trả lời"].apply(lambda x: chunker.split_text(x))

    # Display the chunked results
    for index, row in df_combined.iterrows():
        print(f"\nCâu hỏi: {row['Câu hỏi']}")
        print("Chunks:")
        for i, chunk in enumerate(row["Chunks"]):
            print(f"  Chunk {i+1}: {chunk}")
else:
    print("No data to display.")


Processing file: C:\Users\ungdu\Downloads\Test Chatgpt\Test data.csv
                             Câu hỏi  \
0      Các quả có mùi vị như thế nào   
1  Các quả có hình dáng như thế nào    

                                         Câu trả lời  
0  Quả cam ngon. Quả táo dở. Quả chanh chua. Quả ...  
1  Quả cam có hình tròn. Quả táo có hình tròn, hơ...  

Câu hỏi: Các quả có mùi vị như thế nào
Chunks:
  Chunk 1: Quả cam ngon.
  Chunk 2: Quả táo dở.
  Chunk 3: Quả chanh chua.
  Chunk 4: Quả mít to. Quả mít rất thơm nữa

Câu hỏi: Các quả có hình dáng như thế nào 
Chunks:
  Chunk 1: Quả cam có hình tròn. Quả táo có hình tròn, hơi nhỏ.
  Chunk 2: Quả chanh hình bầu dục.
  Chunk 3: Quả mít to dài có vỏ xù xì.
  Chunk 4: Quả mít có thể lấy gỗ
